In [38]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
from scipy.stats import permutation_test, false_discovery_control, spearmanr
import pybedtools
import gene_expression

from warnings import filterwarnings
filterwarnings("ignore", category=pd.errors.DtypeWarning)
filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

#set up paths
#=========================
workdir = '/home/AD/rkgadde/L1IP'
annotation = f'{workdir}/hg38_data/annotations/gencode.v46.basic.annotation.sorted.genes.gff3'
celltypes = f'{workdir}/celltypes.txt'

genedir = f'{workdir}/gene_data'
figdir = f'{workdir}/results/CZI/plots'

# Load in data

In [45]:
#load in mes
#comb_df = pd.read_csv('/cndd3/dburrows/DATA/me_polymorphisms/analysis/comb_polymorphic_me.csv', sep='\t', index_col=0)
#load in gene expression
cpm_path = '/cndd2/jchien/project/CZI_human/pseudobulk_rna/logcpm/*_combinesample_log2cpm+1.tsv.gz'
suffix = '_combinesample_log2cpm+1.tsv.gz'
cpm = gene_expression.get_expressed_gene_cpm(cpm_path, suffix, 5, 3) # CPM > 5 in at least 2 samples
cpm

,sample,gene_id,cpm,celltype
0,AF1,ENSG00000228463,4.014924,L4-5IT_RORB_TSHZ2
1,AF1,ENSG00000230021,6.679229,L4-5IT_RORB_TSHZ2
2,AF1,ENSG00000225630,3.162097,L4-5IT_RORB_TSHZ2
3,AF1,ENSG00000237973,4.509863,L4-5IT_RORB_TSHZ2
4,AF1,ENSG00000248527,5.159480,L4-5IT_RORB_TSHZ2
...,...,...,...,...
2683723,YM3,ENSG00000212907,9.264718,L56NP_TLE4_TSHZ2
2683724,YM3,ENSG00000198886,11.147137,L56NP_TLE4_TSHZ2
2683725,YM3,ENSG00000198786,9.956563,L56NP_TLE4_TSHZ2
2683726,YM3,ENSG00000198695,8.790963,L56NP_TLE4_TSHZ2


In [46]:
len(cpm[cpm['celltype'] == 'L4-5IT_RORB_TSHZ2']['gene_id'].unique())

14651

In [4]:
#process mes into joint bed file
#process l1
me_type = 'L1'
abs_file = f'{workdir}/mC_data/CZI/type/vars/all_{me_type}_abs.tsv'
ins_file = f'{workdir}/mC_data/CZI/type/vars/all_{me_type}_ins.tsv'
abs_df = pd.read_csv(abs_file, sep='\t', usecols=[0,1,2,3,4,5,6,7,8], names=['chrom','start','end','id', 'length', 'strand', 'class', 'het', 'hom'])[1:]
abs_df['me_type'] = 'absence'
ins_df = pd.read_csv(ins_file, sep='\t', usecols=[0,1,2,3,4,5,6,7,8],  names=['chrom','start','end','id', 'length', 'strand', 'class', 'het', 'hom'])[1:]
ins_df['me_type'] = 'insertion'
l1_df = pd.concat([abs_df, ins_df])
l1_df['class'] = 'l1'

#process alu
me_type = 'Alu'
abs_file = f'{workdir}/mC_data/CZI/type/vars/all_{me_type}_abs.tsv'
ins_file = f'{workdir}/mC_data/CZI/type/vars/all_{me_type}_ins.tsv'
abs_df = pd.read_csv(abs_file, sep='\t', usecols=[0,1,2,3,4,5,6,7,8], names=['chrom','start','end','id', 'length', 'strand', 'class', 'het', 'hom'])[1:]
abs_df['me_type'] = 'absence'
ins_df = pd.read_csv(ins_file, sep='\t', usecols=[0,1,2,3,4,5,6, 7,8],  names=['chrom','start','end','id', 'length', 'strand', 'class', 'het', 'hom'])[1:]
ins_df['me_type'] = 'insertion'
alu_df = pd.concat([abs_df, ins_df])
alu_df['class'] = 'alu'

#add labels
comb_df = pd.concat([l1_df, alu_df])
comb_df['start'] = comb_df['start'].astype(int)
comb_df['end'] = comb_df['end'].astype(int)
comb_df['length'] = comb_df['end'] - comb_df['start']
comb_df['het'] = comb_df['het'].fillna('NaN').astype(str)
comb_df['hom'] = comb_df['hom'].fillna('NaN').astype(str)
comb_df.reset_index(drop=True, inplace=True)
het_bool = np.asarray([True if 'NaN' not in i else False for i in comb_df['het']])
hom_bool = np.asarray([True if 'NaN' not in i else False for i in comb_df['hom']])
het_count = np.asarray([len(i.split(',')) for i in comb_df['het']])
hom_count = np.asarray([len(i.split(',')) for i in comb_df['hom']])
ratio = het_count/hom_count
#set to NaN and inf
ratio[(het_bool == False) & (hom_bool==False)] = np.nan
ratio[(het_bool == True) & (hom_bool==False)] = np.inf
ratio[(het_bool == False) & (hom_bool==True)] = -1*np.inf
comb_df['het_over_hom'] = ratio

# label as majority het or hom, if >2x
genotype = np.empty(len(comb_df)).astype(str)
genotype[ratio >= 2] = 'het'
genotype[ratio <= 0.5] = 'hom'
genotype[(ratio <= 2) & (ratio >= 0.5)] = 'mixed'
genotype[np.isnan(ratio)] = 'NaN'
comb_df['genotype'] = genotype
#label as truncated or full length
trunc = np.empty(len(comb_df), dtype=object)
l1_mask = comb_df['class'] == 'l1'
trunc[l1_mask & (comb_df['length'] > 5500)] = 'full_length'
trunc[l1_mask & (comb_df['length'] <= 5500)] = 'truncated'
alu_mask = comb_df['class'] == 'alu'
trunc[alu_mask & (comb_df['length'] > 280)] = 'full_length'
trunc[alu_mask & (comb_df['length'] <= 280)] = 'truncated'
comb_df['insertion_category'] = trunc

comb_df.to_csv('/cndd3/dburrows/DATA/me_polymorphisms/analysis/comb_polymorphic_me.csv', sep='\t', header=True)

In [ ]:
%%bash
#Get bed file with gene names
#==================================
cd /cndd3/dburrows/DATA/annotations/gencode/
#Split up 4th column into gene id and type
# awk 'BEGIN{OFS="\t"} {split($4, a, "_"); $4 = a[1]; $5 = a[2]; print}' gencode.v44.annotation.hg38.genelabels.bed > tmp_split.bed
# mv tmp_split.bed gencode.v44.annotation.hg38.genelabels.bed

#remove duplicated gene loci (exons from isoforms) that have the same chromosome, start+end position gene_label, and strand
# awk '!seen[$1,$2,$3,$5,$6]++' gencode.v44.annotation.hg38.genelabels.bed > tmp_seen.bed
# mv tmp_seen.bed gencode.v44.annotation.hg38.genelabels.unique.bed
awk -F'\t' '{
    # Extract necessary fields from the 9th column (attributes)
    match($9, /gene_id=([^;]+)/, gene_id);
    match($9, /gene_name=([^;]+)/, gene_name);
    match($9, /gene_type=([^;]+)/, gene_type);

    # Rearrange and print the columns
    print $1, $4, $5, $2, $3, $7, gene_id[1], gene_name[1], gene_type[1];
}' OFS="\t" gencode.v46.basic.annotation.sorted.genes.gff3 > gencode.v46.basic.annotation.sorted.genes.bed

In [30]:
# find intersection of mes with genes
#bedconvert and load annotations
me_bt = pybedtools.BedTool.from_dataframe(comb_df)
gene_df = pd.read_csv('/cndd3/dburrows/DATA/annotations/gencode/gencode.v46.basic.annotation.sorted.genes.bed', sep='\t',header=None)
gene_bt = pybedtools.BedTool.from_dataframe(gene_df)

#compute intersection
me_gene = gene_bt.intersect(me_bt, f=0.01, wo=True) 
me_gene = me_gene.to_dataframe(disable_auto_names=True, header=None)
me_gene.rename(columns={0: 'chr', 1: 'gene_start', 2: 'gene_end', 5: 'gene_strand',
                        6: 'gene_id', 7: 'gene_name', 8: 'gene_type', 10: 'me_start',
                       11: 'me_end', 12: 'me_id', 14: 'me_strand', 15: 'me_class', 16: 'het',
                       17: 'hom', 18: 'me_type', 19: 'het_hom_ratio', 20:'genotype', 
                       21: 'insertion_category', 22: 'length'}, inplace=True)
me_gene.drop(columns=np.asarray(me_gene.columns[(np.asarray([type(i) for i in me_gene.columns]) == int)]), inplace=True)

In [ ]:
# find intersection with genomic regions

#compute intersection
l1_cgi = l1_bt.intersect(cgi_bt, f=0.3, wo=True) 
alu_cgi = alu_bt.intersect(cgi_bt, f=0.3, wo=True) 
alu_all = alu_bt.intersect(all_bt, f=0.3, wo=True) 
l1_all = all_bt.intersect(l1_bt, f=0.3, wo=True) 

l1_cgi_df = l1_cgi.to_dataframe(disable_auto_names=True, header=None)
l1_cgi_df.rename(columns={16: 'region', 9: 'me_type', 11: 'genotype', 12: 'insertion_category', 7: 'het', 8: 'hom', 4: 'length'}, inplace=True)
alu_cgi_df = alu_cgi.to_dataframe(disable_auto_names=True, header=None)
alu_cgi_df.rename(columns={16: 'region', 9: 'me_type', 11: 'genotype', 12: 'insertion_category', 7: 'het', 8: 'hom', 4: 'length'}, inplace=True)
alu_all_df = alu_all.to_dataframe(disable_auto_names=True, header=None)
alu_all_df.rename(columns={16: 'region', 9: 'me_type', 11: 'genotype', 12: 'insertion_category', 7: 'het', 8: 'hom', 4: 'length'}, inplace=True)
l1_all_df = l1_all.to_dataframe(disable_auto_names=True, header=None)
l1_all_df.rename(columns={3: 'region', 15: 'me_type', 17: 'genotype', 18: 'insertion_category', 13: 'het', 14: 'hom', 10: 'length'}, inplace=True)

# eQTL analysis

In [36]:
comb_df

,chrom,start,end,id,length,strand,class,het,hom,me_type,het_over_hom,genotype,insertion_category
0,chr1,56365451,56369289,CZI_abs_1543,3838,+,l1,AF3,"AM1,AM2,AM3,YF2,YM3",absence,0.200000,hom,truncated
1,chr1,65443954,65444209,CZI_abs_1545,255,-,l1,YM2,"AF1,YM1,YM3",absence,0.333333,hom,truncated
2,chr1,80939086,80945263,CZI_abs_1546,6177,-,l1,NaN,"AF2,AF3,AM1,YF1,YM1,YM3",absence,-inf,hom,full_length
3,chr1,82660292,82661886,CZI_abs_1547,1594,+,l1,NaN,"AF1,AF2,AM2,YF1",absence,-inf,hom,truncated
4,chr1,86275302,86276014,CZI_abs_1548,712,+,l1,NaN,"AF1,AM1,YF1,YM1,YM2,YM3",absence,-inf,hom,truncated
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,chr9,124088197,124088200,CZI_ins_4331,3,-,alu,"AF1,AF3,AM1,YF1,YF2,YM2","AF2,AM2,YM3",insertion,2.000000,mixed,truncated
4997,chr9,126217880,126217897,CZI_ins_4333,17,+,alu,"AM1,YM1","AF1,AF2,AF3,YF2,YM2",insertion,0.400000,hom,truncated
4998,chr9,127834834,127834845,CZI_ins_4334,11,-,alu,"AF1,AF2,AF3,AM1,AM2,AM3,YF2,YM1,YM2",YM3,insertion,9.000000,het,truncated
4999,chr9,129426079,129426094,CZI_ins_4335,15,-,alu,NaN,"AF1,AF2,AF3,AM1,AM2,AM3,YF1,YF2,YM1,YM2,YM3",insertion,-inf,hom,truncated


In [37]:
me_gene[]

,chr,gene_start,gene_end,gene_strand,gene_id,gene_name,gene_type,me_start,me_end,me_id,me_strand,me_class,het,hom,me_type,het_hom_ratio,genotype,insertion_category,length
0,chr1,24066774,24083565,+,ENSG00000230703.1,MYOM3-AS1,lncRNA,24076520,24076812,CZI_abs_7,-,alu,"AF1,AF2,AF3,AM1,YF1,YM1,YM2,YM3","AM3,YF2",absence,4.0,het,full_length,292
1,chr1,55915603,55944980,-,ENSG00000230250.3,LINC01753,lncRNA,55935619,55935930,CZI_abs_19,+,alu,.,AF1,absence,-inf,hom,full_length,311
2,chr1,56154545,56477687,-,ENSG00000260971.4,ENSG00000260971,lncRNA,56365451,56369289,CZI_abs_1543,+,l1,AF3,"AM1,AM2,AM3,YF2,YM3",absence,0.2,hom,truncated,3838
3,chr1,56173433,56524495,-,ENSG00000284686.1,ENSG00000284686,protein_coding,56365451,56369289,CZI_abs_1543,+,l1,AF3,"AM1,AM2,AM3,YF2,YM3",absence,0.2,hom,truncated,3838
4,chr1,57605847,57606206,-,ENSG00000236888.1,RPS20P5,processed_pseudogene,57606184,57606197,CZI_ins_818,-,alu,"AM3,YF1","AF1,AF2,AF3,AM1,AM2,YM1,YM3",insertion,0.2857142857142857,hom,truncated,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,chr8,143636019,143656418,+,ENSG00000183309.12,ZNF623,protein_coding,143641445,143641764,CZI_abs_1430,-,alu,.,"AF2,AM3,YM1,YM2",absence,-inf,hom,full_length,319
123,chr8,144772224,144787348,-,ENSG00000196378.14,ZNF34,protein_coding,144776325,144776913,CZI_abs_1433,+,alu,"AF1,AF2,AF3,AM1,AM2,AM3,YF2,YM2,YM3",YM1,absence,9.0,het,full_length,588
124,chr9,35791003,35809732,+,ENSG00000159899.16,NPR2,protein_coding,35803110,35803402,CZI_abs_1464,-,alu,"AF1,AM1,AM2,YF1,YF2,YM2,YM3","AF2,AF3,AM3",absence,2.333333333333333,het,full_length,292
125,chr9,73042745,73056528,+,ENSG00000228024.1,CYP1D1P,unprocessed_pseudogene,73054613,73054880,CZI_abs_1473,+,alu,.,YM1,absence,-inf,hom,truncated,267


In [ ]:
#subset to directly overlapping genes first



In [ ]:
#write code to test association with each gene, including covariates, linreg



In [ ]:
#select genes to test, within certain range, reduce FDR, run

In [ ]:
#visualise significant effects, by celltype, etc.

# metheQTL analysis